<center><h1>2. Vainilla CNNs</h1></center>

In [1]:
# Necessary modules
import pandas as pd
import numpy as np

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.optimizers import SGD

Using Theano backend.


In [2]:
# Get the data
data = pd.read_csv(filepath_or_buffer="../data/face_image_project/fold_0_data.txt", sep="\t")
#data = pd.get_dummies(data, columns=["gender"])
data["gender"][data["gender"] == "f"] = 0
data["gender"][data["gender"] == "m"] = 1
data["gender"][data["gender"] == "u"] = 0

/Library/Python/2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Library/Python/2.7/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Library/Python/2.7/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [3]:
def rowToImgPath(row):
    userid = row["user_id"]
    faceid = row["face_id"]
    original_img = row["original_image"]
    imgPath = "../data/face_image_project/aligned/%s/landmark_aligned_face.%d.%s" % (userid, faceid, original_img)
    return imgPath

def loadXY(data):
    """ BE INCREDIBLY CAREFUL... The data is large - you could run out of memory and start thrashing.
    It is highly advised to load a small subset at a time (100-200 rows)
    """ 
    X = np.zeros((len(data), 816, 816, 3))
    #Y = data[["gender_f", "gender_m", "gender_u"]]
    Y = data["gender"]
    for i in range(len(data)):
        row = data.iloc[i]
        img = load_img(rowToImgPath(row))
        imgArray = img_to_array(img)
        X[i] = imgArray
    return X, Y

def generateXY(data, size=5):
    """ From Keras fit_generator API: The generator is expected to loop over its data indefinitely. """
    while True:
        start, end = 0, size
        while end <= len(data):
            #print start, end
            yield loadXY(data[start: end])
            start, end =  start + size, end + size

In [4]:
# Building a vainilla CNN
vainilla_cnn = Sequential()
vainilla_cnn.add(Convolution2D(32, 3, 3, input_shape=(816,816,3)))
vainilla_cnn.add(Activation('relu'))
vainilla_cnn.add(MaxPooling2D(pool_size=(2, 2)))
vainilla_cnn.add(Convolution2D(64, 3, 3))
vainilla_cnn.add(Activation('relu'))
vainilla_cnn.add(Convolution2D(64, 3, 3))
vainilla_cnn.add(Activation('relu'))
vainilla_cnn.add(MaxPooling2D(pool_size=(2, 2)))
vainilla_cnn.add(Flatten())
vainilla_cnn.add(Dense(output_dim=100, input_dim=500))
vainilla_cnn.add(Activation("relu"))
vainilla_cnn.add(Dense(output_dim=200, input_dim=500))
vainilla_cnn.add(Activation("relu"))
vainilla_cnn.add(Dense(output_dim=1))
vainilla_cnn.add(Activation("sigmoid"))

In [5]:
# Compile
sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
vainilla_cnn.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [7]:
# Train
vainilla_cnn.fit_generator(generateXY(data[:20]), samples_per_epoch=2, nb_epoch=20)

Epoch 1/20
5/2 [===========================================================================] - 28s - loss: 6.3770 - acc: 0.6000
Epoch 2/20
5/2 [===========================================================================] - 28s - loss: 12.7539 - acc: 0.2000
Epoch 3/20
5/2 [===========================================================================] - 26s - loss: 6.3770 - acc: 0.6000
Epoch 4/20
5/2 [===========================================================================] - 25s - loss: 6.3770 - acc: 0.6000
Epoch 5/20
5/2 [===========================================================================] - 22s - loss: 6.3770 - acc: 0.6000
Epoch 6/20
5/2 [===========================================================================] - 21s - loss: 12.7539 - acc: 0.2000
Epoch 7/20
5/2 [===========================================================================] - 21s - loss: 6.3770 - acc: 0.6000
Epoch 8/20
5/2 [===========================================================================] - 22s - l

/Library/Python/2.7/site-packages/keras/engine/training.py:1462: UserWarning: Epoch comprised more than `samples_per_epoch` samples, which might affect learning results. Set `samples_per_epoch` correctly to avoid this warning.
  warnings.warn('Epoch comprised more than '
